# Fisher formalism to forecast kSZ SNR.

### Parameters constrained: 6 \$\Lambda{\rm CDM}$ + A$_{\rm kSZ}$.

#### Noise + foreground residuals: tSZ-free x CIB-free cross-ILC from, for example, publish/data/ilc_weights_residuals_agora_fg_model.npy

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np, sys, os, scipy as sc, warnings
sys.path.append('modules/')
import tools, ilc, fisher_tools
import matplotlib.cbook
import scipy.ndimage as ndimage
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)

#%pylab notebook
%matplotlib inline
from pylab import *


In [3]:
rcParams['figure.dpi'] = 150
rcParams['figure.facecolor'] = 'white'


In [4]:
#get the ILC residuals first
which_fg_model = 'agora'

fname = 'publish/data/ilc_weights_residuals_%s_fg_model.npy' %(which_fg_model)
ilc_dict = np.load(fname, allow_pickle = True).item()
print(ilc_dict.keys())

#get experiment Nl_dict
nl_TP_dict = ilc_dict['nl_TP_dict']

#different types of ILC considered
ilc_keyname_arr = ilc_dict['ilc_keyname_arr']
print(ilc_keyname_arr)

#ILC weights
weights_dict = ilc_dict['weights']
print(weights_dict.keys())

#total ILC residuals
total_ilc_residuals_dict = ilc_dict['total_ilc_residuals']
print(total_ilc_residuals_dict.keys())

if (1): #with CCAT-prime
    fname_withccatp = 'publish/data/ilc_weights_residuals_%s_fg_model_withccatphfnoisescaledby5.0.npy' %(which_fg_model)
    ilc_dict_withccatp = np.load(fname_withccatp, allow_pickle = True).item()

    #get experiment Nl_dict
    nl_TP_dict_withccatp = ilc_dict_withccatp['nl_TP_dict']

    #total ILC residuals
    total_ilc_residuals_dict_withccatp = ilc_dict_withccatp['total_ilc_residuals']
    print(total_ilc_residuals_dict.keys())
    
    ##print(ilc_dict_withccatp.keys())

#el range
els = ilc_dict['el']

dict_keys(['weights', 'total_ilc_residuals', 'cib_plus_tsz_residuals', 'cib_residuals', 'tsz_residuals', 'noise_residuals', 'radio_residuals', 'nl_TP_dict', 'cl_signal_plus_noise', 'ilc_keyname_arr', 'el'])
['mv' 'tszfree' 'cibfree' 'mvxcibfree' 'mvxtszfree' 'tszfreexcibfree']
dict_keys(['s4_wide', 's4_deep', 'spt3g', 'spt4', 'so_baseline', 'so_goal'])
dict_keys(['s4_wide', 's4_deep', 'spt3g', 'spt4', 'so_baseline', 'so_goal'])
dict_keys(['s4_wide', 's4_deep', 'spt3g', 'spt4', 'so_baseline', 'so_goal'])


In [5]:
#experiment names and fsky values
exp_specs_dict = {
                  's4_wide': ['S4-Wide', 0.57], 
                  's4_wide_withccatp': ['S4-Wide + FYST', 0.57], 
                  's4_deep': ['S4-Ultra Deep', 0.03], 
                  'spt3g': ['SPT-3G', 0.03],
                  'spt4': ['SPT-3G+SPT-4', 0.03],
                  'so_baseline': ['SO-Baseline', 0.4],
                  'so_baseline_withccatp': ['SO-Baseline + FYST', 0.4], 
                  'so_goal': ['SO-Goal', 0.4],
                  'so_goal_withccatp': ['SO-Goal + FYST', 0.4], 
                  }

#ILC keynames
ilc_keyname_dict = {'mv':'MV', 'tszfree': 'tSZ-free', 'cibfree': 'CIB-free'}

In [6]:
#some variables for Fisher forecasting
#expname = 's4_wide'
#fsky = exp_specs_dict[expname][1]
planck_cosmo_version = 2015
camb_folder = 'publish/data/CAMB/planck_%s/' %(planck_cosmo_version)
which_spectra = 'lensed'
reqd_ilc_keyname_1 = 'tszfree'
reqd_ilc_keyname_2 = 'cibfree'
if reqd_ilc_keyname_1 != reqd_ilc_keyname_2:
    reqd_ilc_keyname_12 = '%sx%s' %(reqd_ilc_keyname_1, reqd_ilc_keyname_2)
else:
    reqd_ilc_keyname_12 = reqd_ilc_keyname_1

#ksz-related stuffs.
dl_ksz_amp_total = 3. #D_{\ell}^{\rm kSZ} = 3 uK^2 
Aksz = 1. #Amplitude of kSZ.
Aksz_step = Aksz / 100. #stpe for derivative calculation

#parameters for constraining/fixing, power spectra to be used, \ell ranges and priors.
pspectra_to_use = ['TT', 'EE', 'TE']
min_l_pol, max_l_pol = 30, 5000
min_l_temp = 30
max_l_temp = 3000
fix_params = ['mnu', 'ws', 'neff']# but curretnly nothing to fix as we only have a 6+1(neff) LCDM model
prior_dic = {'tau':0.007}#, 'Akszh': dl_ksz_amp_homo/10.}#02}#02} #Planck tau prior

In [7]:
#CMB stuffs
#get fiducual LCDM power spectra computed using CAMB
print('get fiducual LCDM power spectra computed using CAMB')
camb_fname = '%s/cmb_spectra_%s.txt' %(camb_folder, which_spectra)
cl_camb = np.loadtxt(camb_fname)
el_camb = cl_camb[:,0] 
cl_camb_tt = cl_camb[:,1]
dl_fac_camb = el_camb * (el_camb+1)/2/np.pi
cl_dic = {}
cl_dic['TT'] = cl_camb[:,1]
cl_dic['EE'] = cl_camb[:,2]
cl_dic['TE'] = cl_camb[:,3]

#get derivatives of CMB power spectrum for different LCDM parameters.
#They are already computed and stored.
print('get/read derivatives')
camb_deriv_fname = '%s/cmb_spectra_derivs_%s.npy' %(camb_folder, which_spectra)
cl_deriv_dic_tmp = np.load(camb_deriv_fname, allow_pickle = 1).item()
cl_deriv_dic = {}
param_names = []
for p in sorted( cl_deriv_dic_tmp ):
    if p == 'ell': continue    
    cl_deriv_dic[p]={}
    if planck_cosmo_version == 2018:
        cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p]['TT']
        cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p]['EE']
        cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p]['TE']
    else:
        cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p][0]
        cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p][1]
        cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p][2]
    
    if p == 'As' and planck_cosmo_version == 2015:
        cl_deriv_dic[p]['TT'] *= 1e9
        cl_deriv_dic[p]['EE'] *= 1e9
        cl_deriv_dic[p]['TE'] *= 1e9
        
    param_names.append( p )
    
#interpolate CAMB spectra / derivatives on the desired els.
for which_spec in cl_dic:
    cl_dic[which_spec] = np.interp(els, el_camb, cl_dic[which_spec])
for p in sorted( param_names ):
    for which_spec in cl_deriv_dic[p]:
        cl_deriv_dic[p][which_spec] = np.interp(els, el_camb, cl_deriv_dic[p][which_spec])

get fiducual LCDM power spectra computed using CAMB
get/read derivatives


In [8]:
if (0): #compare 2015 and 2018 derivatives
    planck_cosmo_version_arr = [2015, 2018]
    parent_cl_deriv_dic = {}
    for planck_cosmo_version in planck_cosmo_version_arr:
        camb_deriv_fname = 'publish/data/CAMB/planck_%s/cmb_spectra_derivs_%s.npy' %(planck_cosmo_version, which_spectra)
        cl_deriv_dic_tmp = np.load(camb_deriv_fname, allow_pickle = 1).item()
        cl_deriv_dic = {}
        param_names = []
        for p in sorted( cl_deriv_dic_tmp ):
            if p == 'ell': continue
            cl_deriv_dic[p]={}
            if planck_cosmo_version == 2018:
                cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p]['TT']
                cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p]['EE']
                cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p]['TE']
            else:
                cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p][0]
                cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p][1]
                cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p][2]

            if p == 'As' and planck_cosmo_version == 2015:
                cl_deriv_dic[p]['TT'] *= 1e9
                cl_deriv_dic[p]['EE'] *= 1e9
                cl_deriv_dic[p]['TE'] *= 1e9

            param_names.append( p )
        parent_cl_deriv_dic[planck_cosmo_version] = cl_deriv_dic
        print(param_names)
        
    for p in ['As', 'mnu', 'neff', 'ns', 'ombh2', 'omch2', 'tau', 'thetastar']:
    #for p in ['thetastar']:
        clf()
        ax = subplot(111, yscale = 'log')
        which_spec_dict = {'TT': 'navy', 'EE': 'darkgreen', 'TE': 'darkred'}
        for which_spec in which_spec_dict:
            der2015 = parent_cl_deriv_dic[2015][p][which_spec]
            der2018 = parent_cl_deriv_dic[2018][p][which_spec]
            plot(abs(der2018), color = which_spec_dict[which_spec], ls = '-')
            plot(abs(der2015), color = which_spec_dict[which_spec], ls = '--', lw = 4.)
        show(); sys.exit()
            
        
    

In [9]:
#kSZ stuffs
print('get kSZ spectrum and derivatives')
"""
Assumptions:
1. We assume that the derivatives of cl_kSZ w.r.t LCDM parameters is zero.
    (A) This is not 100 per cent true.
2. We also assume that the derivatives of CMB power spectrum w.r.t AkSZ is zero.
    (A) This is a reasonable assumption.
"""

#get kSZ spectrum now: Flat D_{\ell}^{\rm kSZ} = 3 uK^2 
cl_ksz = fisher_tools.get_cl_ksz(els, Aksz = Aksz, dl_ksz_amp_total = dl_ksz_amp_total)

#add kSZ to cl_dic['TT']
cl_dic['TT'] += cl_ksz

#get deriviatives for Aksz
cl_ksz_low = fisher_tools.get_cl_ksz(els, Aksz = Aksz - Aksz_step, dl_ksz_amp_total = dl_ksz_amp_total)
cl_ksz_high = fisher_tools.get_cl_ksz(els, Aksz = Aksz + Aksz_step, dl_ksz_amp_total = dl_ksz_amp_total)
cl_deriv_dic['Aksz'] = {}
cl_deriv_dic['Aksz']['TT'] = (cl_ksz_high-cl_ksz_low)/( 2 * Aksz_step)
cl_deriv_dic['Aksz']['EE'] = cl_deriv_dic['Aksz']['TT'] * 0.
cl_deriv_dic['Aksz']['TE'] = cl_deriv_dic['Aksz']['TT'] * 0.

#add Aksz to param_name
if 'Aksz' not in param_names:
    param_names.append('Aksz')
    
print(param_names)

get kSZ spectrum and derivatives
['As', 'mnu', 'neff', 'ns', 'ombh2', 'omch2', 'tau', 'thetastar', 'Aksz']


In [10]:
def get_temperature_ilc_residuals(expname): #Temperature ILC residuals for Nl
    print('\tget ILC residuals for Nl')
    if expname.find('withccatp')>-1:
        dict_to_use = total_ilc_residuals_dict_withccatp
        expname_to_use = expname.replace('_withccatp', '')
    else:
        dict_to_use = total_ilc_residuals_dict
        expname_to_use = expname
    els, total_ilc_residual_mv = dict_to_use[expname_to_use]['mv']
    els, total_ilc_residual_1 = dict_to_use[expname_to_use][reqd_ilc_keyname_1]
    els, total_ilc_residual_2 = dict_to_use[expname_to_use][reqd_ilc_keyname_2]
    els, total_ilc_residual_12 = dict_to_use[expname_to_use][reqd_ilc_keyname_12]
    
    return els, total_ilc_residual_mv, total_ilc_residual_1, total_ilc_residual_2, total_ilc_residual_12
    
def get_polarisation_mvilc_residuals(expname):
    #get MV-ILC for polarisation
    """
    Note that foregrounds are assumed to be unpolarised here.
    So this should simply be a MV noise estimate after taking beams into account.
    """
    dict_for_ilc = {}
    if expname.find('withccatp')>-1:
        expname_to_use = expname.replace('_withccatp', '')
        dict_for_ilc['EE'] = nl_TP_dict_withccatp[expname_to_use]['P']
    else:
        dict_for_ilc['EE'] = nl_TP_dict[expname]['P']
    bands = get_exp_bands(expname)
    mvilc_pol_residuals, mvilc_pol_weights = ilc.get_mvilc_residual_and_weights(bands, els, dict_for_ilc)
    mvilc_pol_residuals = mvilc_pol_residuals[0]

    if (0):#expname == 's4_wide': #show plot for MV-ILC for pol and compare that will noise.
        clf()
        fsval = 14
        band_color_dict = {95: 'navy', 150: 'darkgreen', 220: 'goldenrod', 285: 'orangered', 345: 'darkred', 
                            '410': 'hotpink', 850: 'black'}
        ax=subplot(111, yscale = 'log')
        noise_arr = []
        for (nu1, nu2) in dict_for_ilc['EE']:
            if nu1 != nu2: continue
            curr_nl = dict_for_ilc['EE'][(nu1, nu2)]

            plot(els, curr_nl, color = band_color_dict[nu1], label = r'%s GHz' %(nu1))
            noise_arr.append(curr_nl)

        #MV-ILC for pol
        plot(els, mvilc_pol_residuals, color = 'black', label = r'MV-ILC')

        if (0): #simple MV noise for pol as a sanity check.
            noise_arr = np.asarray(noise_arr)
            mv_noise_pol = ( np.sum(noise_arr**-2, axis = 0) )**-0.5
            plot(els, mv_noise_pol, color = 'hotpink', lw = 2., ls = '-.', label = r'MV noise for pol.')

        legend(loc = 1, fontsize = fsval - 6, ncol = 5)
        xlabel(r'Multipole $\ell$', fontsize = fsval)
        ylabel(r'Polarisation noise: $C_{\ell}$ [$\mu$K$^{2}$]', fontsize = fsval-2)
        xlim(0., 7000.); ylim(1e-7, .01)
        title_str = r'%s: Polarisation noise + MV-ILC' %(exp_specs_dict[expname][0])
        title(title_str, fontsize = fsval)
        show()
        
    return els, mvilc_pol_residuals

def wrapper_get_delta_cl(els, cl_dic, fsky, 
                            total_ilc_residual_1, total_ilc_residual_2, total_ilc_residual_12, 
                            mvilc_pol_residuals,
                           ):    
    #get delta_Cl using Knox formula.
    nl11_dic = {}
    nl11_dic['TT'] = total_ilc_residual_1
    nl11_dic['EE'] = mvilc_pol_residuals
    nl11_dic['TE'] = np.copy(total_ilc_residual_1) * 0.

    nl22_dic = {}
    nl22_dic['TT'] = total_ilc_residual_2
    nl22_dic['EE'] = mvilc_pol_residuals
    nl22_dic['TE'] = np.copy(total_ilc_residual_2) * 0.

    nl12_dic = {}
    nl12_dic['TT'] = total_ilc_residual_12
    nl12_dic['EE'] = mvilc_pol_residuals
    nl12_dic['TE'] = np.copy(total_ilc_residual_12) * 0.

    delta_cl_dic = fisher_tools.get_knox_errors_parent(els, cl_dic, nl11_dic, fsky, nl22_dic = nl22_dic, nl12_dic = nl12_dic)
    #print(delta_cl_dic.keys())

    if (1): #MV ILC
        nl_mv_dic = {}
        nl_mv_dic['TT'] = total_ilc_residual_mv
        nl_mv_dic['EE'] = mvilc_pol_residuals
        nl_mv_dic['TE'] = np.copy(total_ilc_residual_mv) * 0.
        delta_cl_dic_mv = fisher_tools.get_knox_errors_parent(els, cl_dic, nl_mv_dic, fsky)
        #print(delta_cl_dic_mv.keys()) 
    
    return delta_cl_dic, delta_cl_dic_mv

def get_exp_bands(expname):
    if expname in ['s4_wide', 's4_deep', 'so_baseline', 'so_goal']:        
        bands = [95, 150, 220, 285]
    elif expname == 'spt3g':
        bands = [95, 150, 220]#, 600, 857]
    elif expname == 'spt4':
        bands = [95, 150, 220, 285, 345]
    elif expname in ['s4_wide_withccatp', 'so_baseline_withccatp', 'so_goal_withccatp']: 
        bands = [95, 150, 220, 285, 410, 850]
        
    return bands
    

In [11]:
expname_arr = ['spt3g', 'spt4', 'so_baseline', 'so_baseline_withccatp', 'so_goal', 'so_goal_withccatp', 's4_wide', 's4_deep']
for expname in expname_arr:
    
    print('\n\nExperiment = %s' %(expname))
    
    bands = get_exp_bands(expname)

    #get MV polarisation ILC residuals
    els, mvilc_pol_residuals = get_polarisation_mvilc_residuals(expname)
    dl_fac = els * (els+1)/2/np.pi

    #get temperature ILC residuals
    els, total_ilc_residual_mv, total_ilc_residual_1, total_ilc_residual_2, total_ilc_residual_12 = get_temperature_ilc_residuals(expname)
    if (0):#expname == 's4_wide': #show ILC residuals, CMB, and kSZ spectra
        clf()
        fsval = 14
        ax = subplot(111, yscale = 'log')
        plot(els, dl_fac * cl_dic['TT'], label = r'CMB: TT', lw = 2., color = 'gray')
        plot(els, dl_fac * cl_ksz, label = r'kSZ', lw = 2., color = 'purple')
        plot(els, dl_fac * total_ilc_residual_mv, label = r'ILC: MV', color = 'darkgreen')
        plot(els, dl_fac * total_ilc_residual_1, label = r'ILC: %s' %(ilc_keyname_dict[reqd_ilc_keyname_1]), color = 'goldenrod')
        plot(els, dl_fac * total_ilc_residual_2, label = r'ILC: %s' %(ilc_keyname_dict[reqd_ilc_keyname_2]), color = 'orangered')
        if (1): #cross-ILC
            plot(els, dl_fac * total_ilc_residual_12, lw = 1.5, label = r'ILC: %s x %s' %(ilc_keyname_dict[reqd_ilc_keyname_1], ilc_keyname_dict[reqd_ilc_keyname_2]), color = 'darkred')
            plot(els, dl_fac * -total_ilc_residual_12, lw = 1.5, alpha = 0.3, color = 'darkred', label = r'ILC: %s x %s (Negative)' %(ilc_keyname_dict[reqd_ilc_keyname_1], ilc_keyname_dict[reqd_ilc_keyname_2]))

        xlabel(r'Multipole $\ell$', fontsize = fsval)
        ylabel(r'$D_{\ell}$ [$\mu$K$^{2}$]', fontsize = fsval)
        xlim(0., 5000.); ylim(1., 1e4)
        legend(loc = 1, fontsize = fsval-4)
        title(r'%s' %(exp_specs_dict[expname][0]), fontsize = fsval)
        show(); 

    
    #get delta_cl
    expname_str, fsky = exp_specs_dict[expname]
    delta_cl_dic, delta_cl_dic_mv = wrapper_get_delta_cl(els, cl_dic, fsky, 
                            total_ilc_residual_1, total_ilc_residual_2, total_ilc_residual_12, 
                            mvilc_pol_residuals,
                           )
    
    if (0):#expname == 's4_wide': #show CMB and kSZ spectra; and delta_cl for ILC.
        clf()
        fsval = 14
        ax = subplot(111, yscale = 'log')
        plot(els, dl_fac * cl_dic['TT'], label = r'CMB: TT', lw = 2., color = 'gray')
        plot(els, dl_fac * cl_ksz, label = r'kSZ', lw = 2., color = 'purple')

        delta_cl = delta_cl_dic['TT']
        errorbar(els, dl_fac * cl_ksz, yerr = dl_fac * delta_cl, color = 'black', label = 'Cross-ILC')

        delta_cl_mv = delta_cl_dic_mv['TT']
        errorbar(els, dl_fac * cl_ksz, yerr = dl_fac * delta_cl_mv, color = 'lightgreen', label = 'MV')

        if (0):
            tmp = fisher_tools.get_knox_errors(els, cl_dic['TT'] + total_ilc_residual_mv, fsky, cl22 = None, cl12 = None)
            tmp2 = fisher_tools.get_knox_errors(els, cl_dic['TT'] + total_ilc_residual_1, fsky, cl22 = cl_dic['TT'] + total_ilc_residual_2, cl12 = cl_dic['TT'] + total_ilc_residual_12)
            errorbar(els, dl_fac * cl_ksz, yerr = dl_fac * tmp2, color = 'goldenrod' )
            errorbar(els, dl_fac * cl_ksz, yerr = dl_fac * tmp, color = 'red' )

        xlabel(r'Multipole $\ell$', fontsize = fsval)
        ylabel(r'$D_{\ell}$ [$\mu$K$^{2}$]', fontsize = fsval)
        xlim(0., 5000.); ylim(1., 1e4)
        legend(loc = 1, fontsize = fsval-4)
        title(r'%s: Temperature' %(exp_specs_dict[expname][0]), fontsize = fsval)
        show(); ##sys.exit()

    #get Fisher / COV matrices
    desired_param = 'Aksz'
    print('\tObtain Fisher matrix and parameter constraints.')
    #max_l_temp_arr = [2000, 3000, 3500, 4000, 4500, 5000]
    max_l_temp_arr = [3000, 3500, 4000, 4500]#, 5000]
    delta_cl_dic_arr = [delta_cl_dic_mv, delta_cl_dic]
    delta_cl_dic_str_arr = ['MV', 'cross-ILC']
    latex_str = ''
    for curr_delta_cl_dic, curr_delta_cl_dic_str in zip(delta_cl_dic_arr, delta_cl_dic_str_arr):
        print ('\n')
        #print(curr_delta_cl_dic_str)
        if curr_delta_cl_dic_str == 'MV':
            max_l_temp_arr = [3000]
        else:
             max_l_temp_arr = [3000, 3500, 4000, 4500]

        
        for max_l_temp in max_l_temp_arr:
            F_mat = fisher_tools.get_fisher_matrix(els, cl_deriv_dic, curr_delta_cl_dic, param_names, pspectra_to_use = pspectra_to_use,\
                        min_l_temp = min_l_temp, max_l_temp = max_l_temp, min_l_pol = min_l_pol, max_l_pol = max_l_pol)

            #fix desired parameters
            F_mat, param_names = fisher_tools.fix_params(F_mat, param_names, fix_params)

            cov_mat = np.linalg.inv(F_mat)
            param_names = np.asarray(param_names)
            pind = np.where(param_names == desired_param)[0][0]
            pcntr1, pcntr2 = pind, pind
            cov_inds_to_extract = [(pcntr1, pcntr1), (pcntr1, pcntr2), (pcntr2, pcntr1), (pcntr2, pcntr2)]
            cov_extract = np.asarray( [cov_mat[ii] for ii in cov_inds_to_extract] ).reshape((2,2))
            sigma_Aksz = cov_extract[0,0]**0.5

            snr_Aksz = Aksz/sigma_Aksz
            print( '\t\tILC = %s; lmax = %s; AkSZ SNR = %.2f' %(curr_delta_cl_dic_str, max_l_temp, snr_Aksz) )
            latex_str = '%s & %.2f' %(latex_str, snr_Aksz)
    latex_str = latex_str.strip(' & ')
    print(latex_str); ##sys.exit()




Experiment = spt3g
	get ILC residuals for Nl
	Obtain Fisher matrix and parameter constraints.


		ILC = MV; lmax = 3000; AkSZ SNR = 8.86


		ILC = cross-ILC; lmax = 3000; AkSZ SNR = 7.80
		ILC = cross-ILC; lmax = 3500; AkSZ SNR = 12.46
		ILC = cross-ILC; lmax = 4000; AkSZ SNR = 16.07
		ILC = cross-ILC; lmax = 4500; AkSZ SNR = 18.61
8.86 & 7.80 & 12.46 & 16.07 & 18.61


Experiment = spt4
	get ILC residuals for Nl
	Obtain Fisher matrix and parameter constraints.


		ILC = MV; lmax = 3000; AkSZ SNR = 9.96


		ILC = cross-ILC; lmax = 3000; AkSZ SNR = 9.72
		ILC = cross-ILC; lmax = 3500; AkSZ SNR = 18.51
		ILC = cross-ILC; lmax = 4000; AkSZ SNR = 27.13
		ILC = cross-ILC; lmax = 4500; AkSZ SNR = 34.05
9.96 & 9.72 & 18.51 & 27.13 & 34.05


Experiment = so_baseline
	get ILC residuals for Nl
	Obtain Fisher matrix and parameter constraints.


		ILC = MV; lmax = 3000; AkSZ SNR = 20.20


		ILC = cross-ILC; lmax = 3000; AkSZ SNR = 16.70
		ILC = cross-ILC; lmax = 3500; AkSZ SNR = 25.73
		ILC = cro

	get fisher
MV
2000 6.071909036803356
3000 34.78097901982975
3500 52.353487814177605
4000 67.33569202370403
4500 84.62577025270568
5000 106.24571440568799
cross-ILC
2000 5.823830880165398
3000 31.088912853493213
3500 45.56383650397107
4000 56.773136656497314
4500 66.50673093488548
5000 75.03595771291796


In [286]:
#get Fisher / COV matrices with all 6 LCDM parameters fixed.
print('\tget fisher')
max_l_temp_arr = [2000, 3000, 3500, 4000, 4500, 5000]
delta_cl_dic_arr = [delta_cl_dic_mv, delta_cl_dic]
delta_cl_dic_str_arr = ['MV', 'cross-ILC']
for curr_delta_cl_dic, curr_delta_cl_dic_str in zip(delta_cl_dic_arr, delta_cl_dic_str_arr):
    print(curr_delta_cl_dic_str)
    
    for max_l_temp in max_l_temp_arr:
        F_mat = tools.get_fisher_matrix(els, cl_deriv_dic, curr_delta_cl_dic, param_names, pspectra_to_use = pspectra_to_use,\
                    min_l_temp = min_l_temp, max_l_temp = max_l_temp, min_l_pol = min_l_pol, max_l_pol = max_l_pol)

        fix_params = param_names[:-1]
        F_mat_mod = np.copy(F_mat)
        F_mat_mod, param_names_mod = tools.fix_params(F_mat_mod, param_names, fix_params)

        cov_mat = np.linalg.inv(F_mat_mod)
        sigma_Aksz = np.sqrt( cov_mat )
        snr_Aksz = Aksz/sigma_Aksz
        print(max_l_temp, snr_Aksz )
    

	get fisher
MV
2000 [[7.00101462]]
3000 [[49.87142162]]
3500 [[101.87620878]]
4000 [[165.06043112]]
4500 [[223.63333003]]
5000 [[272.26309535]]
cross-ILC
2000 [[6.91039183]]
3000 [[43.34957391]]
3500 [[74.1123125]]
4000 [[99.95877861]]
4500 [[118.21883196]]
5000 [[130.55687139]]
